In [1]:
import jsonlines
import pandas as pd
import emoji
from soynlp.normalizer import repeat_normalize

In [2]:
from tqdm import tqdm
import selenium
from selenium import webdriver
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import urllib
import re

# Preprocess

In [4]:
def replace_masks(file_in, file_out):
    """
    &name& -> <name>
    url encoding시 &를 넣으면 제대로 들어가지 않으므로 꺾쇠로 대체
    """
    lines = []
    pattern = re.compile(r'&([a-z]+)&') # &name&, &affiliation&, ...
    count = 0
    with jsonlines.open(file_in) as f:
        for line in f:
            sentence = line['sentence_form']
            if pattern.search(sentence):
                sentence = pattern.sub(r'<\1>',sentence)
                count += 1
            line['sentence_form'] = sentence
            lines.append(line)
    
    with jsonlines.open(file_out, 'w') as f:
        f.write_all(lines)
    print("Substitued: ", count)

In [10]:
# (입력 파일명, 출력 파일명)
replace_masks('./Dataset/nikluge-sa-2022-test.jsonl', './Dataset/test_preprocess.jsonl')

Substitued:  20


In [11]:
def preprocess_sentences(file_in, file_out):
    lines = []
    with jsonlines.open(file_in) as f:
        for line in f:
            sentence = line['sentence_form']
            emojis = [e.chars for e in emoji.analyze(sentence)]

            if '♥' not in emojis: # ♥가 들어간 문장만 남기고 replace (test dataset에 있는 특수문자만 남김)
                sentence = emoji.replace_emoji(sentence, '')

            sentence = repeat_normalize(sentence, num_repeats=2) # 반복 어구 축약

            line['sentence_form'] = sentence
            lines.append(line)

    with jsonlines.open(file_out, 'w') as f:
        f.write_all(lines)

In [12]:
# (입력 파일명, 출력 파일명)
preprocess_sentences('./Dataset/test_preprocess.jsonl', './Dataset/test_preprocess.jsonl')

# Backtranslation

In [3]:
import urllib.parse

def translation(json_data, src_lang, tgt_lang, driver: WebDriver):
    """
    request format: sk={src_lang}&tk={tgt_lang}&st={text}
    """
    
    request_url = 'https://papago.naver.com/?sk={}&tk={}&st={}'
    is_formal = True
    with tqdm(json_data, total=len(json_data), desc=f'{src_lang}-{tgt_lang} Translation') as pbar:
        for line in pbar:
            text = line['sentence_form']
            try:
                text = urllib.parse.quote(text) # 특수문자로 인한 encoding
                driver.get(request_url.format(src_lang, tgt_lang, text))
                driver.implicitly_wait(60)
                time.sleep(1.0)

                if tgt_lang == 'ko' and is_formal: # 한국어로 번역할 경우 높임말 해제 (only for en, ja, cn)
                    driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[1]/div[3]/div/div[6]/div/button').click()
                    is_formal = False
                    time.sleep(1.5)
                text = driver.find_element(By.XPATH, '//*[@id="txtTarget"]').text # translation result

            except: # 오류 발생 시 get 방식이 아닌 직접 텍스트를 입력하도록 시도
                driver.get(request_url.format(src_lang, tgt_lang, ''))
                time.sleep(1.5)
                driver.find_element(By.XPATH, '//*[@id="txtSource"]').send_keys(text)
                time.sleep(1.5)
                text = driver.find_element(By.XPATH, '//*[@id="txtTarget"]').text
            line['sentence_form'] = text
            if text == "":
                print("N/A")

    return json_data


In [4]:
# def translation(text_data, src_lang, tgt_lang, driver: WebDriver):
#     """
#     request format:
#     ?sk={src_lang}&tk={tgt_lang}&st={text}
#     """
    
#     request_url = 'https://papago.naver.com/?sk={}&tk={}'
#     driver.get(request_url.format(src_lang, tgt_lang))
#     time.sleep(1.5)
#     # 높임말 해제
#     if tgt_lang == 'ko':
#         driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[1]/div[3]/div/div[6]/div/button').click()
#         disable_formal = True
#         time.sleep(1.5)
    
#     trans_list = []
#     with tqdm(text_data, total=len(text_data), desc=f'{src_lang}-{tgt_lang} Translation') as pbar:
#         for text in pbar:
#             driver.find_element(By.XPATH, '//*[@id="txtSource"]').send_keys(text)
#             time.sleep(2.0)
#             trans = driver.find_element(By.XPATH, '//*[@id="txtTarget"]').text
#             trans_list.append(trans)
#             driver.find_element(By.XPATH, '//*[@id="sourceEditArea"]/button').click()
#             time.sleep(0.1)

#     return trans_list


In [5]:
def augmentation_backtranslate(tgt_lang, file_in, file_out):
    json_data = []
    with jsonlines.open(file_in, 'r') as f:
        for line in f:
            json_data.append(line)

    print(len(json_data))

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    # options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    forward = translation(json_data, 'ko', tgt_lang, driver)
    backward = translation(forward, tgt_lang, 'ko', driver)

    print(len(backward))

    driver.close()

    # assert len(text_data) == len(backward)

    with jsonlines.open(file_out, 'w') as f:
        f.write_all(backward)

    return forward, backward

In [6]:
# (대상 언어, 입력 파일명, 출력 파일명)
forward, backward = augmentation_backtranslate('en', './Dataset/neg.jsonl', './Dataset/neg_aug.jsonl')

8321


ko-en Translation:   1%|          | 84/8321 [01:28<2:24:36,  1.05s/it]

N/A


ko-en Translation:   1%|          | 86/8321 [01:30<2:24:56,  1.06s/it]

N/A


...

N/A


en-ko Translation:  98%|█████████▊| 8145/8321 [2:22:43<03:05,  1.05s/it]

N/A


en-ko Translation: 100%|██████████| 8321/8321 [2:25:48<00:00,  1.05s/it]


8321


# Postprocess


### Check N/A

In [10]:
count = 0
lst = []
with jsonlines.open('./Dataset/neg_aug.jsonl', 'r') as f:
    for line in f:
        if len(line['sentence_form']) == 0: # 오류로 인해 빈 줄이 들어가진 않았는지 확인
            # print (line['sentence_form'])
            lst.append(line['id'])
            count += 1
          
print(count)

75


In [12]:
count = 0
new_lst = []
with jsonlines.open('./Dataset/neg_aug.jsonl', 'r') as f:
    for line in f:
        if len(line['sentence_form']) > 0: # 오류로 인해 빈 줄이 들어가진 않았는지 확인
            # print (line['sentence_form'])
            new_lst.append(line)
            count += 1
          
with jsonlines.open('./Dataset/neg_aug.jsonl', 'w') as f:
    f.write_all(new_lst)

### Merge multiple augmented files

In [14]:
files = [
    './Dataset/train_preprocess.jsonl',
    './Dataset/train_aug_en.jsonl',
    './Dataset/train_aug_jp.jsonl',
    './Dataset/train_aug_cn.jsonl',
    './Dataset/train_aug_es.jsonl',
]

data = []
for i, file in enumerate(files):
    file_data = []
    with jsonlines.open(file, 'r') as f:
        for line in f:
            file_data.append(line)
    data.append(file_data)

with jsonlines.open('./Dataset/train_aug_merged.jsonl', 'w') as f:
    for i in range(len(data[0])):
        for j in range(len(files)):
            f.write(data[j][i])

In [19]:
files = [
    './Dataset/neg.jsonl',
    './Dataset/neg_aug.jsonl',
]

data = []
for i, file in enumerate(files):
    file_data = []
    with jsonlines.open(file, 'r') as f:
        for line in f:
            file_data.append(line)
    data.append(file_data)

with jsonlines.open('./Dataset/neg_merged.jsonl', 'w') as f:
    for i in range(len(files)):
        for j in range(len(data[i])):
            f.write(data[i][j])

In [23]:
files = [
    './Dataset/neg_merged.jsonl',
]

data = []
for i, file in enumerate(files):
    file_data = []
    with jsonlines.open(file, 'r') as f:
        for line in f:
            data.append(line)

sorted(data, key=lambda x: x['id'])
print(data[:10])
# with jsonlines.open('./Dataset/neg_merged_1.jsonl', 'w') as f:
#     for j in range(len(data)):
#         f.write(data[j])

[{'id': 'sentiment-00003', 'sentence_form': '바다쪽으로 내려 가보려고 내려왔는데.. 손가락만 한 바다 바퀴벌레 진짜 많았음.. 여태 본 바다바퀴벌레는 그냥 애기 수준?', 'annotation': [['', [], 'negative']]}, {'id': 'sentiment-00012', 'sentence_form': '아닌가... 맞나.. 는 모르지만 너무 옛기억 아니 맞다', 'annotation': [['', [], 'negative']]}, {'id': 'sentiment-00022', 'sentence_form': '너무 넓어서 너무 아깝기도 한답니다.', 'annotation': [['', [], 'negative']]}, {'id': 'sentiment-00034', 'sentence_form': '단점이라면 주차가 부족하고 주차비가 정말 비싸다.', 'annotation': [['', [], 'negative']]}, {'id': 'sentiment-00038', 'sentence_form': '하지만 찌개가 거기에 못 따라가는 느낌. 그래도 맛은 개인 취향이니까 슴슴 심심한 맛이 내겐 안 맞을 뿐.', 'annotation': [['', [], 'negative']]}, {'id': 'sentiment-00048', 'sentence_form': '나중에 들어보니 이 구간을 택시 기사님들이 상당히 꺼려 하신대요..', 'annotation': [['', [], 'negative']]}, {'id': 'sentiment-00071', 'sentence_form': '지역적으로 레포츠와 축제 음식 등 서울과 수도권 인구의 휴식처로 자리매김하는 중이지만 아직까지 저평가 받고 있는데요', 'annotation': [['', [], 'negative']]}, {'id': 'sentiment-00078', 'sentence_form': '자판기밖에 없어서 카드로는 안되고 꼭 현금이 있어야해요.', 'annotation':